In [1]:
import pandas as pd
import numpy as np

In [2]:

df = pd.read_csv('/home/bowen/Documents/Rod_manipulation/Flexible-Tool/data_2.csv', sep = " ", names = ['name', 'value'])


In [3]:

df.dropna(inplace = True)
df


,name,value
0,a,[0.6152792 0.57624728 0.97183159 0.30227324 0...
1,q,"[[1.0, 0.99996588008, 0.99989695401, 0.9997925..."
2,det_J,"[0.0, 0.0, 0.0, 0.0, -2.53263925567196e-87, 2...."
3,a,[0.42157321 0.04321076 0.28883865 0.53631535 0...
4,q,"[[1.0, 0.99999984126, 0.99999948152, 0.9999988..."
...,...,...
2995,q,"[[1.0, 0.99991970255, 0.99975684995, 0.9995091..."
2996,det_J,"[0.0, 0.0, 0.0, 0.0, 1.4138451607417442e-86, -..."
2997,a,[0.18525751 0.06743073 0.86360801 0.0865571 0...
2998,q,"[[1.0, 0.99999969828, 0.9999992522, 0.99999881..."


In [4]:
df2 = pd.DataFrame()
for r in range(len(df)):
    if r % 3 == 0:
        df0 = df.iloc[r,1:]
        df0.value
        df0 = df0.str.strip('[[[ ')
        df0 = df0.str.strip('[]')
        df0 = df0.value.split()
        df0 = np.array(df0,dtype=float)
        df2 = df2.append(pd.DataFrame(df0))




In [26]:
df2.head(12)
df2.shape

(6000, 1)

In [6]:
df2.shape

(6000, 1)

In [24]:
df3 = pd.DataFrame()
for i in range(len(df)):
    if i % 3 == 1:
        df1 = df.iloc[i,1:]
        df1 = df1.value
        df1 = df1.replace('[','')
        df1 = df1.replace(']','')
        df1 = df1.split(',')
        df1 = np.array(df1,dtype=float)
        df3 = df3.append(pd.DataFrame(df1))

# df1.value
# df1 = list(map(float,df1))


# df1 = np.array(df1)
# for i in range(len(df1)):
#     df1[i] = df1[i].replace('[',' ')
# df1[0]
# arr = np.array([[value.split() for value in range(len(df1))][0] for row in df1], dtype='<f8')
# arr
# df1=np.array([line.split() for line in df1.replace('[','').replace(']]','').split(']')], dtype='<f8')



# type(df1)
# df1 = np.asarray(df1)
# df1 = df1.replace(' ', ',').replace('[', ',').replace(']', ',')
# df1 = df1.value.split()
# type(df1)
# df1 = np.array(df1,dtype=float)
# df3 = df3.append(pd.DataFrame(df1))




In [25]:
df3.shape

(1600000, 1)